<a href="https://colab.research.google.com/github/textnorms/date_text_norm/blob/master/T5_V13_En_relative_dates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Mon Jun 15 11:20:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! rm -rf date*
! git clone https://github.com/textnorms/date_text_norm.git
! cp -r date_text_norm/syntetic_data/ .

! pip install -q num2words transformers
! pip install -q transformers

Cloning into 'date_text_norm'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 302 (delta 53), reused 59 (delta 28), pack-reused 211
Receiving objects: 100% (302/302), 1.45 MiB | 9.53 MiB/s, done.
Resolving deltas: 100% (171/171), done.
     |████████████████████████████████| 102kB 3.6MB/s 
     |████████████████████████████████| 675kB 9.3MB/s 
     |████████████████████████████████| 1.1MB 20.2MB/s 
     |████████████████████████████████| 3.8MB 33.0MB/s 
     |████████████████████████████████| 890kB 55.9MB/s 


# Libs

In [0]:
# Basics
import numpy as np
import pandas as pd
import random

# Synthetic data generator
from syntetic_data import DateTextGenerator
from syntetic_data import RelativeDateTextGenerator

# PyTorch
import torch 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Sklearn
from sklearn.model_selection import train_test_split

# Transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW

# Matplot lib
import matplotlib.pyplot as plt

### Deterministic experiments

In [4]:
manual_seed = 2357 # only primes, cuz I like
def deterministic(rep=True):
    if rep:
        np.random.seed(manual_seed)
        torch.manual_seed(manual_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(manual_seed)
            torch.cuda.manual_seed_all(manual_seed)
        torch.backends.cudnn.enabled = False 
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        print(f'Deterministic experiment, seed: {manual_seed}')
    else:
        print('Random experiment')

deterministic()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Deterministic experiment, seed: 2357
Using device: cuda


# Config constants

In [5]:
# Dataset params
LANGUAGE = 'en'

# Model params
MODEL_SZ = 't5-small' # 't5-base'
TOK = T5Tokenizer.from_pretrained(MODEL_SZ)
MAX_LEN_SRC  = 48
MAX_LEN_TRGT = 12


# Train params
BATCH_SZ = 16
N_EPOCHS = 50
WINDOW   = 7

# Dataset

In [130]:
pd.set_option('display.max_rows',70)

print('Generating absolute and incomplete dates')
dates = DateTextGenerator(start_date='01/01/1921',
                          end_date='31/12/2120',
                          text_noise_rate=0.1,
                          language=LANGUAGE)

print('Generating relative dates')
rel_dates = RelativeDateTextGenerator(n_samples=100, text_noise_rate=0.0,
                                       max_noise_occurences_per_sample=3,
                                       samples_per_method=18,
                                       language=LANGUAGE)

df = dates.generate_date_dataset()
df = df.append(rel_dates.generate_date_dataset(),ignore_index=True)

Generating absolute and incomplete dates
Generating relative dates


In [131]:
df.tail()

,Input Pattern,Noise Type,Input,Target,Target Format
84606,15,N/A,during 100 months,100m,Relative
84607,3,N/A,100 months ago,-100m,Relative
84608,17,N/A,during 100 years,100y,Relative
84609,1,N/A,100 days ago,-100d,Relative
84610,4,N/A,one hundred months ago,-100m,Relative


In [132]:
df['Target Format'].value_counts()

DD/MM/YYYY    73049
MM/YYYY        7200
DD/MM          2562
Relative       1800
Name: Target Format, dtype: int64

In [133]:
df.loc[df['Target Format'] == 'MM/YYYY']

,Input Pattern,Noise Type,Input,Target,Target Format
75611,34,N/A,january.1921,01/1921,MM/YYYY
75612,17,N/A,"01 one thousand, nine hundred and twenty-one",01/1921,MM/YYYY
75613,13,"[remove_char_noise, unexpected_space_noise, lo...",J avary/1g2,01/1921,MM/YYYY
75614,42,N/A,feb/1921,02/1921,MM/YYYY
75615,37,N/A,feb . 1921,02/1921,MM/YYYY
...,...,...,...,...,...
82806,27,"[remove_char_noise, unexpected_space_noise]",nvember - 2 12 0,11/2120,MM/YYYY
82807,19,N/A,"11 two thousand, one hundred and twenty",11/2120,MM/YYYY
82808,22,N/A,12-2120,12/2120,MM/YYYY
82809,15,N/A,"december two thousand, one hundred and twenty",12/2120,MM/YYYY


In [134]:
df.loc[df['Target Format'] == 'Relative']

,Input Pattern,Noise Type,Input,Target,Target Format
82811,7,N/A,for 1 day,1d,Relative
82812,17,N/A,during 1 year,1y,Relative
82813,13,N/A,during 1 day,1d,Relative
82814,3,N/A,1 month ago,-1m,Relative
82815,16,N/A,during one month,1m,Relative
...,...,...,...,...,...
84606,15,N/A,during 100 months,100m,Relative
84607,3,N/A,100 months ago,-100m,Relative
84608,17,N/A,during 100 years,100y,Relative
84609,1,N/A,100 days ago,-100d,Relative


In [135]:
# Removing dates in the defined Target Format
df = df.loc[df['Target Format'] == 'Relative']
print(f'# of patterns: {df["Input Pattern"].nunique()}')
df.head()

# of patterns: 18


,Input Pattern,Noise Type,Input,Target,Target Format
82811,7,N/A,for 1 day,1d,Relative
82812,17,N/A,during 1 year,1y,Relative
82813,13,N/A,during 1 day,1d,Relative
82814,3,N/A,1 month ago,-1m,Relative
82815,16,N/A,during one month,1m,Relative


In [136]:
# Inspecting the relative dates dataset
df.head()

,Input Pattern,Noise Type,Input,Target,Target Format
82811,7,N/A,for 1 day,1d,Relative
82812,17,N/A,during 1 year,1y,Relative
82813,13,N/A,during 1 day,1d,Relative
82814,3,N/A,1 month ago,-1m,Relative
82815,16,N/A,during one month,1m,Relative


## Function to split the dataset

In [137]:
def split_data(df, test_size=0.2, verbose=True):
    l = list(set(df['Input Pattern'].values))
    num_test = int(len(l)*test_size)
    test_methods = [random.randint(1, len(l)) for _ in range(num_test)]
    print(test_methods)
    df_test = df[df['Input Pattern'].isin(test_methods)]
    print(df_test.shape)
    x_test = df_test.Input.values
    y_test = df_test.Target.values

    df_train = df[~df['Input Pattern'].isin(test_methods)]

    x_train, x_val, y_train, y_val = train_test_split(
        df_train.Input.values,
        df_train.Target.values,
        shuffle=True, 
        test_size=test_size,
        random_state=manual_seed
        )
    if verbose:
        print(f'Date types of test set: {test_methods} with len: {len(test_methods)}')
        print(f'x_train: {len(x_train)}  --  y_train: {len(y_train)}\n\
x_val:   {len(x_val)}  --  y_val:   {len(y_val)}\n\
x_test:  {len(x_test)}  --  y_test:  {len(y_test)}')

    return x_train, y_train, x_val, y_val, x_test, y_test

# creating sets
x_train, y_train, x_val, y_val, x_test, y_test = split_data(df, 
                                                            test_size=0.25, 
                                                            verbose=True)

[3, 3, 16, 3]
(200, 5)
Date types of test set: [3, 3, 16, 3] with len: 4
x_train: 1200  --  y_train: 1200
x_val:   400  --  y_val:   400
x_test:  200  --  y_test:  200


In [0]:
class DateDataset(Dataset):
    def __init__(self, data, label, tokenizer, source_max_length, target_max_length):
        self.tokenizer = tokenizer
        self.data = data
        self.label = label
        self.source_max_length = source_max_length
        self.target_max_length = target_max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        source = self.data[idx]
        target = self.label[idx]

        source_tokenized = self.tokenizer.encode_plus(
            f'{source} {self.tokenizer.eos_token}',
            max_length=self.source_max_length,
            pad_to_max_length=True,
            return_tensors='pt')

        target_tokenized = self.tokenizer.encode_plus(
            f'{target} {self.tokenizer.eos_token}',
            max_length=self.target_max_length,
            pad_to_max_length=True,
            return_tensors='pt')

        source_token_ids = source_tokenized['input_ids'].squeeze()
        source_mask = source_tokenized['attention_mask'].squeeze()
        target_token_ids = target_tokenized['input_ids'].squeeze()
        
        return source_token_ids, source_mask, target_token_ids

## Checking the DateDataset class

In [139]:
dataset_debug = DateDataset(
    x_train, 
    y_train,
    TOK,
    MAX_LEN_SRC,
    MAX_LEN_TRGT,
    )

dataloader_checking = DataLoader(
    dataset_debug, 
    batch_size=1, 
    shuffle=True, 
    num_workers=0
    )

source_token_ids, source_mask, target_token_ids = next(iter(dataloader_checking))
print(f'source_token_ids:\n {source_token_ids} --- shape:{source_token_ids.shape}')
print(f'source_mask:\n {source_mask} --- shape:{source_mask.shape}')
print(f'target_token_ids:\n {target_token_ids} --- shape:{target_token_ids.shape}')

source_token_ids:
 tensor([[383, 968, 477,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0]]) --- shape:torch.Size([1, 48])
source_mask:
 tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]) --- shape:torch.Size([1, 48])
target_token_ids:
 tensor([[968,  26,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0]]) --- shape:torch.Size([1, 12])


## Datasets e Dataloaders

In [140]:
# datasets
ds_debug = DateDataset(x_train[:BATCH_SZ], y_train[:BATCH_SZ], TOK, MAX_LEN_SRC, MAX_LEN_TRGT)
ds_train = DateDataset(x_train, y_train, TOK, MAX_LEN_SRC, MAX_LEN_TRGT)
ds_valid = DateDataset(x_val, y_val, TOK, MAX_LEN_SRC, MAX_LEN_TRGT)
ds_test  = DateDataset(x_test, y_test, TOK, MAX_LEN_SRC, MAX_LEN_TRGT)

print('Datasets len:')
print(f'len ds_debug: {len(ds_debug)}')
print(f'len ds_train: {len(ds_train)}')
print(f'len ds_valid: {len(ds_valid)}')
print(f'len ds_test:  {len(ds_test)}')

# dataloaders
dataloaders = {
    'debug': DataLoader(
         ds_debug,
         batch_size=BATCH_SZ,
         shuffle=True,
         num_workers=2,
         pin_memory=True),
    'train': DataLoader(
         ds_train,
         batch_size=BATCH_SZ,
         shuffle=True,
         num_workers=2,
         pin_memory=True),
    'valid': DataLoader(
         ds_valid,
         batch_size=BATCH_SZ,
         shuffle=False,
         num_workers=2,
         pin_memory=True),
    'test': DataLoader(
         ds_test,
         batch_size=BATCH_SZ,
         shuffle=False,
         num_workers=2,
         pin_memory=True),
               }
# sanity check
print('\nDataloaders len (in batch):')
dl_sizes = {x: len(dataloaders[x]) for x in dataloaders.keys()}; dl_sizes

Datasets len:
len ds_debug: 16
len ds_train: 1200
len ds_valid: 400
len ds_test:  200

Dataloaders len (in batch):


{'debug': 1, 'test': 13, 'train': 75, 'valid': 25}

In [0]:
# testando o dataloader 
source_token_ids, source_mask, target_token_ids = next(iter(dataloaders['debug']))

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_SZ)
    
    def forward(self, token_ids, att_mask, labels):
        outputs = self.model.forward(
            input_ids=token_ids, 
            attention_mask=att_mask,
            lm_labels=labels
            )
        return outputs[0] # loss
    
    @torch.no_grad()    
    def generate(self, token_ids, att_mask, max_len_target):
        predict = self.model.generate(
            input_ids=token_ids, 
            attention_mask=att_mask,
            max_length=max_len_target
            )
        return predict
    
    @torch.no_grad()  
    def generate_example(self, text_input, tokenizer, max_len_source=MAX_LEN_SRC):

        self.model.eval()
        
        example_tokenized = tokenizer.encode_plus(
            f'{text_input} {tokenizer.eos_token}',
            max_length=max_len_source,
            pad_to_max_length=True,
            return_tensors='pt')
            
        example_token_ids = example_tokenized['input_ids']
        example_mask = example_tokenized['attention_mask']

        predicted_example = self.model.generate(
            input_ids=example_token_ids.to(device), 
            attention_mask=example_mask.to(device),
            max_length=MAX_LEN_TRGT
            )

        self.model.train()

        out_text = [tokenizer.decode(text) for text in predicted_example]
        
        return out_text

## Train and evaluation functions

In [0]:
# acc metric for text inputs
def acc_in_text(trues, preds): 
    acc = []
    for d in zip(trues, preds):
        if d[0] == d[1]:
            acc.append(1)
        else:
            acc.append(0)
    return acc # bool

def train(model, device, train_loader, optimizer):
    loss_train = []
    model.train()
    for source_token_ids, source_mask, target_token_ids in train_loader:
        optimizer.zero_grad()
        loss = model(
            source_token_ids.to(device), 
            source_mask.to(device), 
            target_token_ids.to(device)
            )
        
        loss_train.append(loss.item())
        loss.backward()
        optimizer.step()
    
    train_losses = sum(loss_train) / len(loss_train)
  
    return train_losses

def evaluate_fn(model, device, val_loader, max_len=MAX_LEN_TRGT):
    loss_val, all_acc, all_preds, all_trues = [], [], [], []
    model.eval()
    for source_token_ids, source_mask, target_token_ids in val_loader:
        predicted_ids = model.generate(
            source_token_ids.to(device), 
            source_mask.to(device),
            max_len
            )
        
        preds = [TOK.decode(t) for t in predicted_ids]
        trues = [TOK.decode(t) for t in target_token_ids]
        acc = acc_in_text(trues, preds)
        all_acc.extend(acc)
        all_trues.extend(trues)
        all_preds.extend(preds)
        
        # val loss   
        loss = model(
        source_token_ids.to(device), 
        source_mask.to(device), 
        target_token_ids.to(device)
        )
        loss_val.append(loss.item())
    
    val_losses = sum(loss_val) / len(loss_val)
    
    return val_losses, np.array(all_acc).mean(), all_trues, all_preds

# Overfit in one batch 
- dataloader debug

In [0]:
overfit = False

if overfit:

    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    deterministic() 

    model = Net().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    
    # -----------------------------------------------------------------------------
    start.record()
    for step in range(1, 1001):
        samp = random.randint(0, BATCH_SZ-WINDOW) # to show random trues and preds
        loss_t = train(model, device, dataloaders['debug'], optimizer)
        loss,acc, trues, preds = evaluate_fn(model, device, dataloaders['debug'])
        if step == 1:
            print(f'[Epoch: {step}/{1000}] |', end=' ')
            print(f'Train Loss: {loss_t:.3f} -- Acc: {acc:.3f}')
        if step % 100 == 0:
            print(f'[Epoch: {step}/{1000}] |', end=' ')
            print(f'Train Loss: {loss_t:.3f} -- Acc: {acc:.3f}')
            print(f'  Trues: {trues[samp:samp+WINDOW]}\n  Preds: {preds[samp:samp+WINDOW]}')
        if acc>= 0.99:
          print('The model has overfitted! Breaking the loop :)')
          break
    end.record()
    torch.cuda.synchronize()    
    # -----------------------------------------------------------------------------

    print(f'Training time: {start.elapsed_time(end)/1000/60 :.3f} min.')
    del model

# Training 

In [147]:
try:
  del model
except:
  print('Model already erased, starting a new one!')
  
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
deterministic() 

model = Net().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

N_EPOCHS = 10

# ---------------------------------------------------------------------------------
start.record()
for step in range(1, N_EPOCHS+1):
    samp = random.randint(0, BATCH_SZ-WINDOW) # to show random trues and preds
    loss_t = train(model, device, dataloaders['train'], optimizer)
    loss_v, acc, trues, preds = evaluate_fn(model, device, dataloaders['valid'])
    print(f'[Epoch: {step}/{N_EPOCHS}] |', end=' ')
    print(f'Train Loss: {loss_t:.3f} -- Valid Loss: {loss_v:.3f} -- Acc: {acc:.3f}')
    print(f'  Trues: {trues[samp:samp+WINDOW]}\n  Preds: {preds[samp:samp+WINDOW]}')

end.record()
torch.cuda.synchronize()    
# ---------------------------------------------------------------------------------

print(f'Training time: {start.elapsed_time(end)/1000/60 :.3f} min.')

Deterministic experiment, seed: 2357
[Epoch: 1/10] | Train Loss: 3.929 -- Valid Loss: 1.031 -- Acc: 0.000
  Trues: ['10y', '63d', '-37m', '-75m', '47m', '-65d', '-4y']
  Preds: ['', '', '', '', '', '', '']
[Epoch: 2/10] | Train Loss: 1.145 -- Valid Loss: 0.223 -- Acc: 0.552
  Trues: ['-65d', '-4y', '-95d', '14y', '26m', '8y', '89d']
  Preds: ['65d', 'Vier ans', '95d', '14y', '26m', '8y', ' ⁇   ⁇   ⁇   ⁇  ⁇  ⁇  ⁇ ']
[Epoch: 3/10] | Train Loss: 0.361 -- Valid Loss: 0.105 -- Acc: 0.677
  Trues: ['63d', '-37m', '-75m', '47m', '-65d', '-4y', '-95d']
  Preds: ['63d', '-37m', '-75m', '47m', '65d', '-4y', '95d']
[Epoch: 4/10] | Train Loss: 0.215 -- Valid Loss: 0.064 -- Acc: 0.810
  Trues: ['-75m', '47m', '-65d', '-4y', '-95d', '14y', '26m']
  Preds: ['-75m', '47m', '-65d', '-4y', '-95d', '-14y', '26m']
[Epoch: 5/10] | Train Loss: 0.145 -- Valid Loss: 0.043 -- Acc: 0.930
  Trues: ['-37m', '-75m', '47m', '-65d', '-4y', '-95d', '14y']
  Preds: ['-37m', '-75m', '47m', '-65d', '-4y', '-95d', '14y']

# Test

In [148]:
# ---------------------------------------------------------------------------------
start.record()

samp = random.randint(0, BATCH_SZ-WINDOW) # to show random trues and preds
loss, acc, trues, preds = evaluate_fn(model, device, dataloaders['test'])
print(f'Loss: {loss:.3f} -- Acc: {acc:.3f}')
print(f' Trues: {trues[samp:samp+WINDOW]}\n  Preds: {preds[samp:samp+WINDOW]}')

end.record()
torch.cuda.synchronize()    
# ---------------------------------------------------------------------------------

print(f'Test time: {start.elapsed_time(end)/1000/60 :.3f} min.')

Loss: 0.012 -- Acc: 0.975
 Trues: ['5m', '-5m', '-6m', '6m', '-7m', '7m', '8m']
  Preds: ['5m', '-5m', '-6m', '6m', '-7m', '7m', '8m']
Test time: 0.020 min.


# The End

